In [ ]:
import matplotlib.pyplot as plt
from skimage import color
from skimage import io
from keras.models import model_from_yaml
import keras
import numpy as np
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from skimage.transform import resize
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_curve
from joblib import dump, load

In [ ]:
# Load encoder
yaml_file = open('encoderc.yaml', 'r') #enter path to encoder yaml file
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model.load_weights("encoderc.h5") #enter path to encoder h5 file
print("Loaded model from disk")

In [ ]:
# Compile training set data
X_train_d = np.zeros((800, 32, 32, 6))
Y_train_d = np.zeros((800, 1))
for i in range(1, 401):
    image = f"data_train_annotation/negative_L/img_negative_L_{i}.jpg" #Pass negative training data
    img = resize(io.imread(image), (32,32))
    x1 = loaded_model(img.reshape(1,32,32,3))
    X_train_d[(i-1) * 2, :, :, 0:3] = img
    X_train_d[(i-1) * 2, :, :, 3:6] = x1[0]


    image = f"data_train_annotation/positive_L/img_positive_L_{i}.jpg" #Pass positive training data
    img = resize(io.imread(image), (32,32))
    x1 = loaded_model(img.reshape(1,32,32,3))
    X_train_d[((i-1) * 2) + 1, :, :, 0:3] = img
    X_train_d[((i-1) * 2) + 1, :, :, 3:6] = x1[0]
    Y_train_d[((i-1) * 2) + 1] = 1

In [ ]:
# Compile validation set data
X_vali_d = np.zeros((800, 32, 32, 6))
Y_vali_d = np.zeros((800, 1))
for i in range(1, 401):
    image = f"data_vali_annotation/negative_L/img_negative_L_{i}.jpg" #Pass negative validation data
    img = resize(io.imread(image), (32,32))
    x1 = loaded_model(img.reshape(1,32,32,3))
    X_vali_d[(i-1) * 2, :, :, 0:3] = img
    X_vali_d[(i-1) * 2, :, :, 3:6] = x1[0]


    image = f"data_vali_annotation/positive_L/img_positive_L_{i}.jpg" #Pass positive validation data
    img = resize(io.imread(image), (32,32))
    x1 = loaded_model(img.reshape(1,32,32,3))
    X_vali_d[((i-1) * 2) + 1, :, :, 0:3] = img
    X_vali_d[((i-1) * 2) + 1, :, :, 3:6] = x1[0]
    Y_vali_d[((i-1) * 2) + 1] = 1

In [ ]:
# Compile test set 
X_test_d = np.zeros((800, 32, 32, 6))
Y_test_d = np.zeros((800, 1))
for i in range(1, 401):
    image = f"data_test_annotation/negative_L/img_negative_L_{i}.jpg" #Pass negative testing data
    img = resize(io.imread(image), (32,32))
    x1 = loaded_model(img.reshape(1,32,32,3))
    X_test_d[(i-1) * 2, :, :, 0:3] = img
    X_test_d[(i-1) * 2, :, :, 3:6] = x1[0]


    image = f"data_test_annotation/positive_L/img_positive_L_{i}.jpg" #Pass positive testing data
    img = resize(io.imread(image), (32,32))
    x1 = loaded_model(img.reshape(1,32,32,3))
    X_test_d[((i-1) * 2) + 1, :, :, 0:3] = img
    X_test_d[((i-1) * 2) + 1, :, :, 3:6] = x1[0]
    Y_test_d[((i-1) * 2) + 1] = 1


In [ ]:
# Flatten data
X_train_d = X_train_d.reshape((800, 32*32*6))
Y_train_d = Y_train_d.reshape(800,)

X_vali_d = X_vali_d.reshape((800, 32*32*6))
Y_vali_d = Y_vali_d.reshape(800, )

X_test_d = X_test_d.reshape((800, 32*32*6))
Y_test_d = Y_test_d.reshape(800, )

1) VANILLA SVM

In [ ]:
# Combine training and validation sets
X_train_f = np.vstack((X_train_d, X_vali_d))
Y_train_f = np.vstack((Y_train_d, Y_vali_d))

In [ ]:
# Training procedure
support_vector_classifier = SVC(C = 9, kernel='rbf')
support_vector_classifier.fit(X_train_f, Y_train_f.reshape(1600,))

# Evaluation
score = support_vector_classifier.score(X_test_d, Y_test_d)
print(score)

b) metrics

In [ ]:
Y_pred_d = support_vector_classifier.predict(X_test_d)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(Y_test_d, Y_pred_d)
print(cm)

In [ ]:
# Precision
pr = precision_score(Y_test_d, Y_pred_d)
print(pr)

In [ ]:
# Recall
pr = recall_score(Y_test_d, Y_pred_d)
print(pr)

In [ ]:
# F1 Score 
f1 = f1_score(Y_test_d, Y_pred_d)
print(f1)

In [ ]:
fpr, tpr, thresholds= roc_curve(Y_test_d, Y_pred_d)

In [ ]:
plt.plot(fpr, tpr)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')

2) PCA SVM

In [ ]:
# PCA
pca = PCA(0.99)
pca.fit(X_train_f)

X_train_PCA = pca.transform(X_train_f)
X_test_PCA = pca.transform(X_test_d)
print(X_train_PCA.shape)

In [ ]:
# Training procedure 
support_vector_classifier = SVC(C = 5.9, kernel='rbf')
support_vector_classifier.fit(X_train_PCA, Y_train_f.reshape(1600,))

#Evaluation
score = support_vector_classifier.score(X_test_PCA, Y_test_d)
print(score)

b) metrics

In [ ]:
Y_pred_d = support_vector_classifier.predict(X_test_PCA)

In [ ]:
# Confusion Matrix
cm = confusion_matrix(Y_test_d, Y_pred_d)
print(cm)

In [ ]:
# Precision
pr = precision_score(Y_test_d, Y_pred_d)
print(pr)

In [ ]:
# Recall
pr = recall_score(Y_test_d, Y_pred_d)
print(pr)

In [ ]:
# F1 Score 
f1 = f1_score(Y_test_d, Y_pred_d)
print(f1)

In [ ]:
fpr, tpr, thresholds= roc_curve(Y_test_d, Y_pred_d)
print(tpr)
print(fpr)

In [ ]:
plt.plot(fpr, tpr)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')